In [2]:
import chainer
import chainer.functions as F
import chainer.links as L
# trianerクラスが新機能として追加
from chainer import training
from chainer.training import extensions
from chainer import Chain, optimizers, serializers

In [3]:
# 自動的にMNISTデータセットをダウンロードし、NumPy配列を $(HOME)/.chainer ディレクトリに保存する。
train, test = chainer.datasets.get_mnist()

# chainer.iterators.SerialIteratorが内部でchainer.dataset.iterator(以下イテレーター)を呼んでいる
# 訓練データセットは毎ループでシャッフルしたい。
train_iter = chainer.iterators.SerialIterator(train, batch_size=100)

# shuffle=Falseを引数に与えることで、シャッフルを無効化できる
# repeat=Falseとしたが、これはすべての要素を見た時に繰り返しが終了することを意味する。
test_iter = chainer.iterators.SerialIterator(test, 100,repeat=False, shuffle=False)

In [3]:
# ネットワークの組み方・ハイパーパラ、メータの設定は変更なし
class MnistModel(chainer.Chain):
    def __init__(self):
        super(MnistModel,self).__init__(
                l1 = L.Linear(784,100),
                l2 = L.Linear(100,100),
                l3 = L.Linear(100,10))

    def __call__(self,x):    
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        return self.l3(h)

In [4]:
class Classifier(Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__(predictor=predictor)

    def __call__(self, x, t):
        # 入力がndarrayだった場合、Variableが自動でラッピング可能
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)
        accuracy = F.accuracy(y, t)
        
        # report()関数は損失と精度の値をtrainerに報告する
        report({'loss': loss, 'accuracy': accuracy}, self)
        return loss

In [5]:
model = L.Classifier(MnistModel())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [6]:
# Trainerを構築する
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (20, 'epoch'), out="result")

In [ ]:
# TrainerクラスのExtentionを用い、結果を可視化する

# Evaluator = 毎ループで、テストデータセットに対する現在のモデルを評価する
trainer.extend(extensions.Evaluator(test_iter, model))
# LogReport = 報告された値を蓄積し、ログファイルを出力ディレクトリに書き出す。
trainer.extend(extensions.LogReport())
# PrintReport = LogReportの中の選択された値をプリントする。
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
# ProgressBar = 進捗状況をバーで可視化する
trainer.extend(extensions.ProgressBar())

trainer.run()  

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
     total [..................................................]  1.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 20 epochs
    207.08 iters/sec. Estimated time to finish: 0:00:56.981610.
     total [#.................................................]  2.50%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 20 epochs
    204.49 iters/sec. Estimated time to finish: 0:00:57.215457.
     total [#.................................................]  3.33%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 20 epochs
    202.34 iters/sec. Estimated time to finish: 0:00:57.329829.
     total [##................................................]  4.17%
this epoch [#########################################.........] 83.33%
       500 iter, 0 epoch / 20 epochs
    20

### その他Extensionが持つ機能

In [ ]:
# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# epochごとにsnapshotをとり、ディレクトリに保存
trainer.extend(extensions.snapshot())
if args.resume:
    # Resume from a snapshot
    chainer.serializers.load_npz(args.resume, trainer)